In [2]:
Hidden_states = ('Healthy', 'Fever') # 隐状态集合
 
Observations_states = ('normal', 'cold', 'dizzy') # 观测状态集合
 
Start_probability = {'Healthy': 0.6, 'Fever': 0.4} # 表示病人第一次到访时医生认为其所处的HMM状态，他唯一知道的是病人倾向于是健康的（可以理解为这是基于一个对大众身体信息的了解得出的初始状态）
 
Hidden_transition_probability = { # 隐马尔可夫链中身体状态的状态转移概率，我们能够看到，当天健康的人，第二天有30%的概率会发烧
   'Healthy' : {'Healthy': 0.7, 'Fever': 0.3},
   'Fever' : {'Healthy': 0.4, 'Fever': 0.6},
   }
 
Hidden_observations_probability = {    # 原来叫emission_probability。这里表示病人每天的感觉的可能性。即，如果他是一个健康人，有50%的可能会感觉正常，40%觉得冷，10%觉得头晕
   'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
   'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6},
   }

# Helps visualize the steps of Viterbi.
def print_dptable(V): # 打印dp矩阵
    print ("    "),
    for i in range(len(V)): 
        print("%7d" % i)
    print()

    for y in V[0].keys():
        print ("%.5s: " % y)
        for t in range(len(V)):
            print ("%.7s" % ("%f" % V[t][y]))
        print()

def viterbi(obs, states, start_p, trans_p, h2o_p): # Viterbi算法
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for y in states:
        V[0][y] = start_p[y] * h2o_p[y][obs[0]]
        path[y] = [y]

    # Run Viterbi for t > 0
    for t in range(1,len(obs)):
        V.append({})
        newpath = {}

        for y in states:
            (prob, state) = max([(V[t-1][y0] * trans_p[y0][y] * h2o_p[y][obs[t]], y0) for y0 in states])
            V[t][y] = prob
            newpath[y] = path[state] + [y]

        # Don't need to remember the old paths
        path = newpath

    print_dptable(V)
    (prob, state) = max([(V[len(obs) - 1][y], y) for y in states])
    return (prob, path[state])

def example():
    return viterbi(Observations_states,
                   Hidden_states,
                   Start_probability,
                   Hidden_transition_probability,
                   Hidden_observations_probability)
print(example())




    
      0
      1
      2

Healt: 
0.30000
0.08400
0.00588

Fever: 
0.04000
0.02700
0.01512

(0.01512, ['Healthy', 'Healthy', 'Fever'])



'''
example print:

      0
      1
      2

Healt:
0.30000
0.08400
0.00588

Fever:
0.04000
0.02700
0.01512

(0.01512, ['Healthy', 'Healthy', 'Fever'])

In [6]:
obs = ('normal', 'cold', 'dizzy')
states = ('Healthy', 'Fever')
start_p = {'Healthy': 0.6, 'Fever': 0.4}
trans_p = {
   'Healthy' : {'Healthy': 0.7, 'Fever': 0.3},
   'Fever' : {'Healthy': 0.4, 'Fever': 0.6}
   }
emit_p = {
   'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
   'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6}
   }

In [10]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    # 这个写的很清晰
    V = [{}]
    # V t st 表示在t天的时候，状态为st的概率。
    for st in states:
        # 初始状态，第0天状态为st
        V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        # 从第一天开始 直到最后一天（最后一个观测状态）
        V.append({})
        for st in states:
            # 每一天都针对每个state做处理，我们要取最大的那个
            max_tr_prob = V[t-1][states[0]]["prob"]*trans_p[states[0]][st] # 首先计算从第0天的到今天的概率，从隐藏状态的第一个候选状态开始计算
            prev_st_selected = states[0]  # 第一天的状态我们也肯定就假设是states[0]
            for prev_st in states[1:]: # 然后对每一个候选状态都做类似的事情，选出最大的那个
                tr_prob = V[t-1][prev_st]["prob"]*trans_p[prev_st][st]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st
                    
            # 其实可以用这个代替以上的操作
            # max_tr_prob, prev_st_selected = max([(V[t-1][states[0]]["prob"]*trans_p[states[0]][st], st) for st in states])
            # 选出使当前概率最大的那个prev_st 即 前一天的状态，并且也知道了当前的概率
            
            # 那么此时当天的最大的概率 就是最大的隐藏状态*对应的发射概率
            max_prob = max_tr_prob * emit_p[st][obs[t]]
            # 将这个值保存到v[t][st]中，即此时里面保存了两个值，一个是当前的最大值，另一个是对应的前一刻的值
            V[t][st] = {"prob": max_prob, "prev": prev_st_selected}
    # 然后打印出对应的dp表
    for line in dptable(V):
        print (line)
    
    # V计算完成了
    opt = []
    max_prob = 0.0
    previous = None
    # Get most probable state and its backtrack 
    # 倒着 把路径给找到输出出来
    for st, data in V[-1].items():
        # 最后一天有多个st，每个st都有相对应的prob，我们取最大的那个作为我们要的st
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st
    
    # Follow the backtrack till the first observation
    # 接着，我们倒着找到一路上的路径
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]

    print ('The steps of states are ' + ' '.join(opt) + ' with highest probability of %s' % max_prob)

def dptable(V):
    # Print a table of steps from dictionary
    yield " ".join(("%12d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V)

In [9]:
viterbi(obs,
        states,
        start_p,
        trans_p,
        emit_p)

           0            1            2
Healthy: 0.30000 0.08400 0.00588
Fever: 0.04000 0.02700 0.01512
The steps of states are Healthy Healthy Fever with highest probability of 0.01512
